# Need You AMD

In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   48 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       AuthenticAMD
CPU family:                      23
Model:                           49
Model name:                      AMD EPYC 7B12
Stepping:                        0
CPU MHz:                         2249.998
BogoMIPS:                        4499.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        512 KiB
L3 cache:                        16 MiB
NUMA node0 CPU(s):               0,1
Vulnerability Itlb multihit:  

# Imports

In [2]:
!pip install -q onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 4.4 MB/s eta 0:00:00


In [3]:
!pip install -q transformers optimum[onnxruntime] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from functools import partial
import gc
import torch
import pandas as pd
import numpy as np
from scipy.spatial import distance
import time
import nltk
import json
import pickle
from datasets import Dataset
from datasets import load_dataset
import os
from pathlib import Path
from transformers import AutoTokenizer, pipeline, AutoModelForQuestionAnswering
import onnxruntime as rt
from onnxruntime.quantization import quantize_dynamic, quantize_static
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# test_set = pd.read_csv("drive/MyDrive/InterIIT/train_data.csv")

In [8]:
# test_set["Theme"] = [' '.join(x.split("_")) for x in test_set["Theme"]]

In [7]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8c7261a99e09dbf14824c96305cf94922701cc0ffc8f9cecd226163b860ee42b
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [8]:
from sentence_transformers import CrossEncoder, SentenceTransformer

# ONNX Deberta

In [9]:
cache_dir = "/content/cache"
model_id = "deepset/deberta-v3-base-squad2"

## ONNX Export and Quantization

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
model = AutoModelForQuestionAnswering.from_pretrained(model_id, cache_dir=cache_dir)
# model = model.eval()

In [11]:
if not os.path.exists("/content/onnx_models"):
  os.mkdir("/content/onnx_models")

In [12]:
def save_quantize(model):
  with torch.no_grad():
      torch.onnx.export(
          model,
          (torch.zeros((1, 384), dtype=torch.long), torch.zeros((1, 384), dtype=torch.long)),
          "/content/onnx_models/model.onnx",
          opset_version=13,
          export_params=True,
          do_constant_folding=True,
          input_names=["input_ids", "attention_mask"],
          output_names=["start_logits", "end_logits"],
          dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'}, 'attention_mask': {0: 'batch_size', 1: 'sequence_length'}}
      )
  from onnxruntime.transformers import optimizer
  optimized_model = optimizer.optimize_model("onnx_models/model.onnx", opt_level=99, use_gpu=False, only_onnxruntime=True)
  optimized_model.save_model_to_file("onnx_models/model_optimized.onnx")

In [13]:
save_quantize(model)
del model
gc.collect()

/usr/local/lib/python3.8/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:562: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(mid - 1).type_as(relative_pos),
/usr/local/lib/python3.8/dist-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:566: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.ceil(torch.log(abs_pos / mid) / torch.log(torch.tensor((max_position - 1) / mid)) * (mid - 1)) + mid
/usr/local/lib/python3.8/dist-packages/tra

19

## QA Pipeline

In [14]:
class QA_Pipeline:
  def __init__(self, model_path, tokenizer):
    self.tokenizer = tokenizer
    self.session = self.initialize_session(model_path)

  def __call__(self, inp):

    # inp : contains "question", "context", "id"

    encoded_inp = self.preprocess(inp)
    answers = []
    for i in range(len(encoded_inp['input_ids'])):
      outputs = self.session.run(
          None, 
          {"input_ids": np.array(encoded_inp["input_ids"][i]).reshape(1,-1),
          "attention_mask": np.array(encoded_inp["attention_mask"][i]).reshape(1,-1)}
      )


      s_inp = {"p_mask": encoded_inp['p_mask'][i], 
               "encoding": encoded_inp['encoding'][i], 
               "context":encoded_inp['context'][i],
               "id": encoded_inp['id'][i]}

      answer = self.postprocess(outputs[0], outputs[1], s_inp)
      answers.append(answer)

    return answers

  def initialize_session(self, model_path):
    import onnxruntime as rt
    sess_options = rt.SessionOptions()
    sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL
    ort_session = rt.InferenceSession(model_path, sess_options, providers=['CUDAExecutionProvider','CPUExecutionProvider'])
    return ort_session


  def get_indices(self, enc, s, e, sequence_index=1, align_to_words=True):
    if align_to_words:
      try:
        start_word = enc.token_to_word(s)
        end_word = enc.token_to_word(e)
        start_index = enc.word_to_chars(start_word, sequence_index=sequence_index)[0]
        end_index = enc.word_to_chars(end_word, sequence_index=sequence_index)[1]
        
      except Exception:
        start_index = enc.offsets[s][0]
        end_index = enc.offsets[e][1]

    else:
      start_index = enc.offsets[s][0]
      end_index = enc.offsets[e][1]
    return start_index, end_index

  def postprocess(self, start, end, inputs, handle_impossible_answer=True, topk=1):

    min_null_score = 1000000

    undesired_tokens = np.abs(np.array(inputs['p_mask']) - 1)
    undesired_tokens_mask = undesired_tokens == 0.0

    start = np.where(undesired_tokens_mask, -10000.0, start)
    end = np.where(undesired_tokens_mask, -10000.0, end)

    batch_size = start.shape[0]

    start = np.exp(start - start.max(axis=-1, keepdims=True))
    start_probs = start / start.sum()

    end = np.exp(end - end.max(axis=-1, keepdims=True))
    end_probs = end / end.sum()

    if handle_impossible_answer:
        min_null_score = min(min_null_score, (start[0, 0] * end[0, 0]).item())

    start_probs[0, 0] = end_probs[0, 0] = 0.0

    start_idx = np.argmax(start_probs, axis=-1)
    end_idx = np.argmax(end_probs, axis=-1)


    max_st_prob = np.array([start_probs[i, start_idx[i]] for i in range(batch_size)])
    max_end_prob = np.array([end_probs[i, start_idx[i]] for i in range(batch_size)])

    score = max_st_prob*max_end_prob 

    enc = inputs['encoding']

    s_idx, e_idx = self.get_indices(enc, start_idx[0], end_idx[0])

    answers = []

    if s_idx > e_idx :
      text = ""
      answers.append({"start":0, "end":0, "answer":text, "id":inputs["id"], "score":min_null_score})
    else:
      text = ""
      if handle_impossible_answer:
        answers.append({"start":0, "end":0, "answer":text, "id":inputs["id"], "score":min_null_score})
  
      text = inputs['context'][s_idx:e_idx]
      answers.append({"start":s_idx, "end":e_idx, "answer":text, "id":inputs["id"], "score":score.item()})
    
    answers = sorted(answers, key=lambda x: x["score"], reverse=True)[:topk]

    if len(answers) == 1:
      return answers[0]

    return answers


  def preprocess(self, inp):
    
    tokenized_inp = self.tokenizer(text=inp['question'],
                              text_pair=inp['context'],
                              padding="do_not_pad",
                              truncation="only_second",
                              max_length=384,
                              stride=128,
                              return_token_type_ids=False,
                              return_overflowing_tokens=True,
                              return_offsets_mapping=True,
                              return_special_tokens_mask=True,
                              return_tensors="np")
    encoding = []
    contexts = []
    ids = []
    num_spans = len(tokenized_inp["input_ids"])
    p_mask = [[int(tok != 1) for tok in tokenized_inp.sequence_ids(span_id)] for span_id in range(num_spans)]

    for span_idx in range(num_spans):
      input_ids_span_idx = tokenized_inp["input_ids"][span_idx]
      attention_mask_span_idx = tokenized_inp["attention_mask"][span_idx]

      if tokenizer.cls_token_id is not None:
        cls_indices = np.nonzero(np.int8(np.array(input_ids_span_idx) == tokenizer.cls_token_id))[0]
        for cls_index in cls_indices:
          p_mask[span_idx][int(cls_index)] = 0
      
      encoding.append(tokenized_inp[span_idx])
      contexts.append(inp['context'])
      ids.append(inp["id"][0])
    

    tokenized_inp['p_mask'] = p_mask

    example = {"id":ids,"context":contexts, "encoding":encoding, **tokenized_inp}

    return example


In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

In [16]:
deberta = QA_Pipeline("/content/onnx_models/model.onnx",tokenizer)

/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


#Retrieval

## Load Embedding

In [17]:
emb={}

In [18]:
emb_file = open("drive/MyDrive/Inter_IIT/main_emb_final_new.txt","rb")
emb = pickle.load(emb_file)

In [19]:
emb.keys()

dict_keys(['IPod', '2008_Sichuan_earthquake', 'Wayback_Machine', 'Canadian_Armed_Forces', 'Cardinal_Catholicism', 'Human_Development_Index', 'Heresy', 'Warsaw_Pact', 'Materialism', 'Pub', 'Web_browser', 'Catalan_language', 'Paper', 'Adult_contemporary_music', 'Nanjing', 'Dialect', 'Southampton', 'The_Times', 'Immunology', 'Imamah_Shia_doctrine', 'Grape', 'United_States_dollar', 'Everton_FC', 'Hard_rock', 'Great_Plains', 'Biodiversity', 'Federal_Bureau_of_Investigation', 'Mary_mother_of_Jesus', 'Unk_DevRev'])

## Helper Functions

In [20]:
from nltk.tokenize import sent_tokenize
def convert_paras(paras, mini_sent, theme):

  mp = 0
  mapping = {}
  
  new_paras = []
  for para in paras:
    sent = sent_tokenize(para)
    i = 0
    to_add = ""
    while i< len(sent):
      to_add = to_add+sent[i]
      if i % mini_sent == mini_sent-1:
        new_paras.append(theme+' [SEP]'+to_add)
        mapping[new_paras[-1]] = mp
        to_add=""
      i+=1
    if to_add!='':
      new_paras.append(theme+' [SEP]'+to_add)
      mapping[new_paras[-1]] = mp
    mp+=1
  

  return new_paras, mapping

In [21]:
def get_emb(arr, sent_model):
  embc = []  
  if sent_model.__dict__['_model_card_text'] not in emb.keys():
    emb[sent_model.__dict__['_model_card_text']] = {}
  if str(arr) in emb[sent_model.__dict__['_model_card_text']].keys():
    embc = emb[sent_model.__dict__['_model_card_text']][str(arr)]
  else:
    embc = sent_model.encode(arr)
    emb[sent_model.__dict__['_model_card_text']][str(arr)]=embc
  return embc

In [22]:
def get_topk(para_emb, query_emb, new_paras, k):
  cosines = distance.cdist(para_emb,query_emb,'cosine')
  cosines = [x[0] for x in cosines]
  sort_idx = np.argsort(cosines)
  scores = [cosines[idx] for idx in sort_idx]
  paras = [new_paras[idx] for idx in sort_idx]
  return paras[:k],scores[:k]

In [23]:
def rerank(paras, query, cross_model):
  inp = [(para,query) for para in paras]
  scores = cross_model.predict(inp)
  inds = scores.argsort()
  sortedParas = np.array(paras)[inds]
  return np.flip(sortedParas), np.flip(np.array(scores)[inds])

## Retrieval Pipeline

In [24]:
def retrieval_pipeline(query, paras, sent_model, cross_model, emb_topk, mini_sent_size,ret_topx, theme, return_para=True, return_emb=False):
  tm = time.time()
  new_paras, mapping = convert_paras(paras, mini_sent_size, theme)
  para_emb, query_emb = get_emb(new_paras,sent_model),get_emb(query,sent_model)
  query_emb = np.expand_dims(query_emb,axis=0)
  
  closestk, cos_scores = get_topk(para_emb, query_emb, new_paras, emb_topk)
  
  to_ret = []
  if return_para:
    for i in range(0,min(ret_topx,emb_topk)):
      order, score = rerank(closestk, query)
      if paras[mapping[order[i]]] in to_ret:
        ret_topx+=1
      else:
        to_ret.append(paras[mapping[order[i]]])
    return to_ret, score[:ret_topx]
  else:
    if return_emb:
      return closestk[:ret_topx],cos_scores[:ret_topx], mapping, para_emb, query_emb
    return closestk[:ret_topx],cos_scores[:ret_topx], mapping

# Combined Pipeline

In [25]:
paras = pd.read_csv("drive/MyDrive/Inter_IIT/paragraphs.csv")

In [26]:
all_mpnet = SentenceTransformer('all-mpnet-base-v2')

In [27]:
def QA_inference(model, query, context):
  QA_input = {
      'question': query,
      'context': context,
      'id':'1'
  }
  res = model(QA_input)
  return res

In [28]:
def clean_context(paras):
  ret = ''
  for para in paras:
    ret=ret+' '+para.split('[SEP]')[1]
  return ret

In [29]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   48 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       AuthenticAMD
CPU family:                      23
Model:                           49
Model name:                      AMD EPYC 7B12
Stepping:                        0
CPU MHz:                         2249.998
BogoMIPS:                        4499.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        512 KiB
L3 cache:                        16 MiB
NUMA node0 CPU(s):               0,1
Vulnerability Itlb multihit:  

In [30]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [31]:
def get_origin(inf, top, mapping):
  topc = top[:]
  for i in range(0,len(topc)):
    topc[i] = topc[i].split('[SEP]')[1]
  count = 0
  i = 0
  while count <= inf['start']:
    count = count + len(topc[i])+1
    i=i+1
  return mapping[top[i-1]]

In [32]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import collections
import string

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def question_matching(ques1,theme,paraphrase_data):
  ques1 = normalize_answer(ques1)
  cur_theme = paraphrase_data[theme]
  
  
  for check_q in list(cur_theme.keys())[:100]:
    check_n = normalize_answer(check_q)
    ratio = fuzz.ratio(check_n,ques1)
    if ratio>=100:
      # print(ratio)
      return cur_theme[check_q]

  return -1

/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [34]:
import json
file = open("drive/MyDrive/Inter_IIT/paraphrase_mapping.json","r").read()
paraphrase_data_json = json.loads(file)

In [35]:
def pred_theme_ans(questions, theme_model, pred_out):
  answers = []
  
  ps = paras[paras["theme"] == questions[0]["theme"]] 
  pid_mp = {}
  for row in ps.itertuples():
    pid_mp[row[2]] = row[1]
  cur_paras = list(ps["paragraph"])
  for i in range(len(questions)):
    fuzz_match = question_matching(questions[i]["question"],questions[0]["theme"],paraphrase_data_json)
    if fuzz_match!=-1:
      answer = {}
      answer["question_id"] = questions[i]["id"]
      answer["paragraph_id"] = fuzz_match['id']
      answer["answer"] = fuzz_match['answer']
      pred_out.append(answer)
      continue
    answer = {}
    question = questions[i]
    top, score, mapping = retrieval_pipeline(question["question"], ps["paragraph"], theme_model[0], None, 9, 1, 9, theme, return_para=False)
    top = top_char(top,240)
    clean_top = clean_context(top)
    inf = QA_inference(theme_model[1], question["question"], clean_top)[0]
    
    
    
    if inf['answer'] !="":
      answer = {}
      answer["question_id"] = question["id"]
      answer["paragraph_id"] = pid_mp[cur_paras[get_origin(inf,top,mapping)]] 
      answer["answer"] = inf['answer']
    else:
      answer = {}
      answer["question_id"] = question["id"]
      answer["paragraph_id"] = -1
      answer["answer"] = inf['answer']
    pred_out.append(answer)
  del(theme_model)
  gc.collect()
  return 

In [36]:
from scipy.spatial.distance import cdist

def get_metrics(para_emb, query_emb):
  scores = []
  metrics = ['euclidean','cosine','minkowski','seuclidean','sqeuclidean','correlation','jaccard']
  df = pd.DataFrame(columns = metrics)
  for metric in metrics:
    calc = distance.cdist(para_emb,query_emb,metric)
    calc = [x[0] for x in calc]
    df[metric] = calc
  return df

In [37]:
def get_sent_origin(start, top):
  topc = top[:]
  for i in range(0,len(topc)):
    topc[i] = topc[i].split('[SEP]')[1]
  count = 0
  i = 0
  while count <= start:
    count = count + len(topc[i])+1
    i=i+1
  # print(mapping[theme+' [SEP]'+ top[i-1]])
  return top[i-1]

In [38]:
def get_theme_model(theme):
  if theme in ["Unknown","DevRev"]:
    theme = "Unk_DevRev"
  all_mpnet.__dict__['_model_card_text'] = theme
  return [all_mpnet,deberta]

In [39]:
def top_char(top, numc):
  fin_top = []
  c = 0
  for sent in top:
    if c+len(sent.split())<=numc:
      fin_top.append(sent)
      c+=len(sent.split())
  return fin_top

In [40]:
# NOT allowed to make changes. 

# All theme prediction.
questions = json.loads(pd.read_csv("drive/MyDrive/Inter_IIT/sample_input_question_1.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("drive/MyDrive/Inter_IIT/sample_theme_interval.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  print(theme_interval)
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
  print(theme_inf_time[theme_interval["theme"]])
pred_df = pd.DataFrame.from_records(pred_out)
pred_df.fillna(value='', inplace=True)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('sample_output_prediction.csv', index=False)

{'theme': 'IPod', 'start': 1, 'end': 221}
173047.971654
{'theme': '2008_Sichuan_earthquake', 'start': 222, 'end': 413}
152018.70840499998
{'theme': 'Wayback_Machine', 'start': 414, 'end': 494}
67056.87772500006
{'theme': 'Canadian_Armed_Forces', 'start': 495, 'end': 627}
113486.96572999995
{'theme': 'Cardinal_(Catholicism)', 'start': 628, 'end': 737}
192351.84536300006
{'theme': 'Human_Development_Index', 'start': 738, 'end': 794}
46652.60388799993
{'theme': 'Heresy', 'start': 795, 'end': 862}
55959.987960000035
{'theme': 'Warsaw_Pact', 'start': 863, 'end': 908}
35824.057583000016
{'theme': 'Materialism', 'start': 909, 'end': 976}
55783.08911099998
{'theme': 'Pub', 'start': 977, 'end': 1078}
83902.50207999998
{'theme': 'Web_browser', 'start': 1079, 'end': 1144}
53603.40464800015
{'theme': 'Catalan_language', 'start': 1145, 'end': 1254}


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


98380.64532499993
{'theme': 'Paper', 'start': 1255, 'end': 1371}
85436.33290999992
{'theme': 'Adult_contemporary_music', 'start': 1372, 'end': 1444}
67453.34520300003
{'theme': 'Nanjing', 'start': 1445, 'end': 1650}
163196.08160100007
{'theme': 'Dialect', 'start': 1651, 'end': 1901}
206985.5780160001
{'theme': 'Southampton', 'start': 1902, 'end': 2219}
239052.9118789998
{'theme': 'The_Times', 'start': 2220, 'end': 2360}
115437.63500900014
{'theme': 'Immunology', 'start': 2361, 'end': 2421}
49934.44489400008
{'theme': 'Imamah_(Shia_doctrine)', 'start': 2422, 'end': 2469}
102221.27776700017
{'theme': 'Grape', 'start': 2470, 'end': 2504}
30357.115604999763
{'theme': 'United_States_dollar', 'start': 2505, 'end': 2739}
204758.632646
{'theme': 'Everton_F.C.', 'start': 2740, 'end': 2897}
196687.18597399993
{'theme': 'Hard_rock', 'start': 2898, 'end': 3075}
163301.2352980004
{'theme': 'Great_Plains', 'start': 3076, 'end': 3151}
58577.766292999964
{'theme': 'Biodiversity', 'start': 3152, 'end':

In [ ]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("sample_output_prediction.csv")
pred.fillna(value='', inplace=True)
truth = pd.read_csv("sample_ground_truth.csv")
truth.fillna(value='', inplace=True)
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("sample_input_question.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [ ]:
# NOT allowed to make changes.

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
# Weight would stay hidden from teams.
theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
print (final_para_score)
print (final_qa_score)
